In [20]:
import requests
from bs4 import BeautifulSoup
import csv

# task 1
url = "https://www.baraasallout.com/test.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
with open('Extract_Text_Data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    header = ["Type", "Content"]
    writer.writerow(header)
    headings = [h.get_text(strip=True) for h in soup.find_all(['h1', 'h2'])]
    paragraphs = [tag.get_text(strip=True) for tag in soup.find_all('p')]
    list_items = [tag.get_text(strip=True) for tag in soup.find_all('li')]
    for heading in headings:
        writer.writerow(["Heading", heading])
    for paragraph in paragraphs:
        writer.writerow(["Paragraph", paragraph])
    for list_item in list_items:
        writer.writerow(["List Item", list_item])

In [21]:
#task 2
with open("Extract_Table_Data.csv", mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    table = soup.find("table")
    headers = [h.get_text(strip=True) for h in table.find_all("th")]
    writer.writerow(headers) 
    if table:
        rows = table.find_all("tr")
        for row in rows[1:]:  # skip the header <th>
            cols = row.find_all("td")
            writer.writerow([col.text.strip() for col in cols])

In [ ]:
#task 3 : extract products info
import json
product_info = []
#<div style="display: flex; justify-content: space-around; margin-top: 20px;">
products = soup.select("div[style*='display: flex;'] > div")

for c in products:
    title = c.find("strong").text.strip() if c.find("strong") else ""
    price = c.find("p", style="color: green;").text.strip() if c.find("p", style="color: green;") else ""
    stock = c.find_all("p")[2].text.strip() if len(c.find_all("p")) > 2 else ""
    button = c.find("button").text.strip() if c.find("button") else ""

    product_info.append({
        "Book Title": title,
        "Price": price,
        "Stock Availability": stock,
        "Button Text": button
    })

with open("Product_Information.json", "w", encoding="utf-8") as jsonfile:
    json.dump(product_info, jsonfile, indent=4)

In [ ]:
#task 4 : extract form details which user fill in contact us
form_details = []
form = soup.find("form")
if form:
    input = form.find_all("input")
    for i in input:
        field_name = i.get("name", "")
        input_type = i.get("type", "")
        default_value = i.get("value", "")

        form_details.append({
            "Field Name": field_name,
            "Input Type": input_type,
            "Default Value": default_value
        })

with open("Form_Details.json", "w", encoding="utf-8") as jsonfile:
    json.dump(form_details, jsonfile, indent=4)

In [ ]:
#task 5 : extract hyperlinks and video link 
links_and_media = {
    "Hyperlinks": [],
    "Video Links": []
}
# extract hyperlinks <a>
for a in soup.find_all("a", href=True):
    links_and_media["Hyperlinks"].append(a["href"])
# extract video links <iframe>
iframe = soup.find("iframe")
if iframe and iframe.get("src"):
    links_and_media["Video Links"].append(iframe["src"])
with open("Links_and_Multimedia.json", "w", encoding="utf-8") as jsonfile:
    json.dump(links_and_media, jsonfile, indent=4)

In [ ]:
#task 6 : extract featured products info
featured_products = []
featured_section = soup.find("div", class_="products")
if featured_section:
    product_cards = featured_section.find_all("div", class_="product-card")
    for c in product_cards:
        product_id = c.get("data-id", "")
        name = c.find("p", class_="name").text.strip() if c.find("p", class_="name") else ""
        price = c.find("p", class_="price", style="display: none;").text.strip() if c.find("p", class_="price", style="display: none;") else ""
        colors = c.find("p", class_="colors").text.strip() if c.find("p", class_="colors") else ""

        featured_products.append({
            "id": product_id,
            "name": name,
            "price": price,
            "colors": colors
        })
with open("Featured_Products.json", "w", encoding="utf-8") as jsonfile:
    json.dump(featured_products, jsonfile, indent=4)